In [ ]:
!pip install gdown
!pip install ipython-autotime

In [ ]:
%load_ext autotime

time: 164 µs (started: 2021-03-03 13:12:03 +00:00)


In [ ]:
!gdown "https://drive.google.com/uc?id=1IATZNB9SCDWkPhU1okCKwf_NMFSC_LSn"

Downloading...
From: https://drive.google.com/uc?id=1IATZNB9SCDWkPhU1okCKwf_NMFSC_LSn
To: /content/collaboration_network_weighted.edgelist
113MB [00:00, 199MB/s]
time: 2.43 s (started: 2021-03-03 13:12:03 +00:00)


In [ ]:
#%%cython
import numpy as np
import networkx as nx
from random import randint
import random
import pandas as pd

time: 776 ms (started: 2021-03-03 13:12:05 +00:00)


# Deep walk By hand

In [ ]:
#%%cython
# Simulates a random walk of length "walk_length" starting from node "node"
def random_walk(G, node, walk_length):
    v = node 
    walk = [v]
    for _ in range(walk_length) :
        neighbors = list(G.neighbors(walk[-1]))
        v = random.choice(list(G.neighbors(node)))
        walk.append(v)
    
    walk = [str(node) for node in walk]
    return walk

# Runs "num_walks" random walks from each node
def generate_walks(G, num_walks, walk_length):
    walks = []

    for _ in range (num_walks):
        All_nodes_p = np.random.permutation(list(G.nodes()))
        for n in All_nodes_p:
            walks.append(random_walk(G,n,walk_length))
    
    return walks

In [ ]:
# Simulates walks and uses the Skipgram model to learn node representations
def deepwalk(G, num_walks, walk_length, n_dim):
    print("Generating walks")
    walks = generate_walks(G, num_walks, walk_length)

    print("Training word2vec")
    model = Word2Vec(size=n_dim, window=8, min_count=0, sg=1, workers=8)#hs=1
    model.build_vocab(walks)
    model.train(walks, total_examples=model.corpus_count, epochs=5)

    return model

time: 6.1 ms (started: 2021-02-25 14:14:37 +00:00)


In [ ]:
# Loads the web graph
G = nx.read_weighted_edgelist('collaboration_network_weighted.edgelist', delimiter=' ', nodetype=int , create_using=nx.Graph())
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

Number of nodes: 231239
Number of edges: 2991383
time: 18.1 s (started: 2021-02-26 08:17:33 +00:00)


In [ ]:
# Hyperparameters
n_dim = 256
n_walks = 100
walk_length = 30
n = G.number_of_nodes() 

time: 10.9 ms (started: 2021-02-25 14:15:11 +00:00)


In [ ]:
from tqdm import tqdm
from gensim.models import Word2Vec
# Extracts a set of random walks from the web graph and feeds them to the Skipgram model
model = deepwalk(G, n_walks, walk_length, n_dim)

Generating walks
Training word2vec
time: 14min 32s (started: 2021-02-25 14:15:13 +00:00)


In [ ]:
nodes = model.wv.index2entity[:n]
vecs = np.empty(shape=(n, n_dim))
print('Genrating vec')
for i in tqdm(range(len(vecs))):
  vecs[i,:] = model.wv[nodes[i]]

 14%|█▍        | 33215/231239 [00:00<00:00, 332143.19it/s]

Genrating vec


100%|██████████| 231239/231239 [00:00<00:00, 337939.25it/s]

time: 700 ms (started: 2021-02-25 14:29:46 +00:00)


In [ ]:
df_z = pd.DataFrame(vecs, index = list(G.nodes))
df_z.index.name = 'authorID'
df_z.to_csv('DeepWalk_128.csv')

time: 79.2 ms (started: 2021-02-25 14:30:30 +00:00)


# Git code

**Need to colne repo from here** ::
https://github.com/shenweichen/GraphEmbedding/tree/7e26f8a0b648cfe0ad3a6f13fd1aced8c1218503

In [ ]:
G = nx.parse_edgelist(open('collaboration_network_weighted.edgelist', 'r'), nodetype=int)

time: 1min 2s (started: 2021-03-03 13:12:16 +00:00)


In [ ]:
# a changer le path to the repo
%cd /content/drive/MyDrive/Polytechnique/P2/Altegrad/Project on h-index regression/GraphEmbedding-master 


/content/drive/.shortcut-targets-by-id/1kS1iQtluaRpEaZCHOindzTWlK9csEstr/Polytechnique/P2/Altegrad/Project on h-index regression/GraphEmbedding-master
time: 10.5 ms (started: 2021-03-03 13:13:18 +00:00)


In [ ]:
!python setup.py install

running install
running bdist_egg
running egg_info
writing ge.egg-info/PKG-INFO
writing dependency_links to ge.egg-info/dependency_links.txt
writing requirements to ge.egg-info/requires.txt
writing top-level names to ge.egg-info/top_level.txt
reading manifest file 'ge.egg-info/SOURCES.txt'
writing manifest file 'ge.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/ge
copying build/lib/ge/walker.py -> build/bdist.linux-x86_64/egg/ge
copying build/lib/ge/classify.py -> build/bdist.linux-x86_64/egg/ge
copying build/lib/ge/__init__.py -> build/bdist.linux-x86_64/egg/ge
copying build/lib/ge/utils.py -> build/bdist.linux-x86_64/egg/ge
copying build/lib/ge/alias.py -> build/bdist.linux-x86_64/egg/ge
creating build/bdist.linux-x86_64/egg/ge/models
copying build/lib/ge/models/__init__.py -> build/bdist.linux-x86_64/egg/ge/models
copying build/lib/ge/models

In [ ]:
#Hyper pram
n_dim = 256
n_walks = 100
walk_length = 30

time: 2.03 ms (started: 2021-02-27 13:06:47 +00:00)


## Deep walk

In [ ]:
# Maps old labels to new labels
new_labels = dict( (i , str(i) ) for i in G.nodes() )
# Relabel nodes
G_str = nx.relabel_nodes(G,new_labels,False)  

time: 37.6 s (started: 2021-02-26 08:18:46 +00:00)


In [ ]:
from ge import DeepWalk
model = DeepWalk(G_str, walk_length=walk_length, num_walks=n_walks, workers=1)#init model
model.train(embed_size=n_dim)# train model
embeddings = model.get_embeddings()# get embedding vectors

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.9min finished


Learning embedding vectors...


In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127
authorID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2811232256,-0.700594,0.536247,-0.702120,-0.154143,1.189856,0.088109,0.109493,0.467330,1.091008,0.125945,-0.067022,0.806139,0.027166,0.742652,-0.297541,0.099469,0.349131,-0.476117,0.508167,0.010630,-0.336774,0.485781,-0.449258,0.313447,-0.220388,-0.235123,0.350160,0.210591,-0.851338,-0.909582,0.377469,-0.397180,-0.112305,-0.730675,0.709452,-0.166814,-0.264570,0.267404,-0.890300,0.209053,...,-0.829969,0.167119,-0.559202,-0.522154,-0.313245,-0.675389,0.666860,0.908852,-0.275173,-0.062528,-0.300244,-0.232704,0.297268,0.061690,-0.253681,0.010111,0.592190,-1.456985,0.149339,0.349093,-0.234244,0.395349,-0.081155,0.425030,0.293423,-0.202995,-0.285919,0.643705,0.071169,1.369186,-0.831508,-0.323459,-0.521527,0.691571,-0.737787,0.299174,-0.310564,0.181778,0.344984,-0.321026
2231369732,0.146430,0.039708,0.145519,-0.110275,0.126038,0.422608,0.294441,-0.204920,0.919955,0.517140,-0.036698,0.166846,0.155180,-0.289016,0.030667,-0.542937,0.498932,0.226399,-0.101905,0.073733,-0.074103,-0.732774,0.500674,0.006040,-0.301969,-0.062110,-0.109131,0.189118,-0.298562,0.060541,0.772046,-0.047499,0.030082,-0.377116,-0.283199,-0.352946,0.650723,0.351094,-0.739227,-0.903983,...,0.167765,0.449066,-0.685457,0.317292,-0.322672,0.265250,0.059109,0.429963,-0.327539,0.131076,-0.630329,0.310579,0.259273,-0.042894,-0.117991,-0.181318,-0.775035,-0.102093,0.098333,-0.675847,1.079068,0.002419,0.380430,-0.192684,-0.005607,0.106166,-0.592483,0.755167,-0.220544,0.361344,0.441071,0.009435,-0.345728,0.268108,0.232140,0.032919,0.655839,-0.086898,0.096106,-0.005643
2138570756,0.578427,0.995748,-0.056903,-0.418689,0.862017,0.463203,0.738235,-1.151097,0.468837,0.559307,-0.192701,-0.154343,0.262031,0.781251,0.492480,-0.119424,0.336151,-0.488268,1.445146,0.349162,0.331278,-0.769405,0.250452,1.162369,-0.099078,0.172803,-0.056218,-0.338600,-0.380206,-0.928179,0.655403,-0.125771,0.945316,-0.851464,0.163893,0.567867,0.636821,-0.153915,0.364191,-0.180532,...,0.994085,0.614035,1.248970,0.927409,-1.778887,0.709334,-1.197690,0.383125,1.043256,-0.029431,-0.598992,0.781046,-1.539336,0.734831,-0.777647,0.556622,-0.241399,-0.942597,0.481439,-0.579484,-0.326054,-0.223943,1.634803,0.766759,-0.803868,0.632717,-0.528895,0.271721,-0.090071,-0.091285,1.348981,0.706690,-0.636818,-0.431727,-0.524515,0.502437,-1.772266,1.459054,0.259837,-0.425322
41943048,0.292996,-0.319963,-0.822594,0.348158,0.296819,0.278393,-0.208899,0.111571,-0.121168,0.157193,-0.616087,-0.528806,-1.183039,0.724731,0.944700,-0.566430,-1.135047,-2.013440,1.506503,0.175919,-0.202859,-0.042361,-0.327237,0.577442,1.086206,-0.225088,-0.444610,-0.405221,0.301496,0.753871,0.380406,0.134494,0.735522,-1.115940,0.502755,-0.234585,-1.905087,0.854919,-0.415732,0.416615,...,-0.813836,1.274979,0.402333,-0.264997,-1.096639,-0.702043,-0.423471,-0.613678,0.048840,0.423995,-0.271779,-0.543791,0.243238,-1.448739,0.060276,0.462705,-0.309543,-2.232011,-0.596301,0.017452,-0.463879,0.308276,0.356301,0.140470,-0.126050,2.036283,-0.809613,-0.364030,1.529073,-0.266029,0.845644,0.820670,0.548721,-0.107398,-0.837809,1.627898,-0.254597,-1.155637,-0.591364,0.106400
2014838794,0.326513,0.198054,0.025147,-0.296216,-0.235460,-0.448570,0.087317,0.138287,0.028561,0.178283,0.315265,-0.389226,0.284661,-0.108501,0.071854,-0.024936,-0.419399,0.059653,0.132076,0.364576,-0.011489,0.371706,0.123921,0.755081,-0.202565,0.372897,-0.312050,-0.258604,-0.117871,0.189429,-0.000510,0.449615,0.332844,-0.556808,-0.393914,0.261120,0.047434,-0.336479,-0.455439,0.318008,...,0.111478,-0.022750,-0.338550,-0.005776,-0.521950,0.558982,-0.385205,-0.388300,0.175309,-0.220432,0.002835,0.017041,-0.129841,-0.197787,0.060393,0.566102,-0.186861,-

time: 115 ms (started: 2021-02-25 21:12:20 +00:00)


In [ ]:
df = pd.DataFrame.from_dict(embeddings, orient='index')#pd.DataFrame(embeddings)#, index = list(G.nodes))
df.index.name = 'authorID'
df.to_csv('DeepWalk2_128.csv')

time: 1min 1s (started: 2021-02-25 21:13:18 +00:00)


## LINE


In [ ]:
from ge import LINE

model = LINE(G,embedding_size=n_dim,order='second') #init model,order can be ['first','second','all']
model.train(batch_size=512,epochs=20,verbose=2)# train model
embeddings = model.get_embeddings()# get embedding vectors

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
34949/34949 - 5429s - loss: 0.3201
Epoch 2/20
34949/34949 - 5382s - loss: 0.2059
Epoch 3/20
34949/34949 - 5369s - loss: 0.1836
Epoch 4/20
34949/34949 - 5400s - loss: 0.1674
Epoch 5/20
34949/34949 - 5359s - loss: 0.1537
Epoch 6/20
34949/34949 - 5299s - loss: 0.1420
Epoch 7/20


In [ ]:
embeddings = model.get_embeddings()

In [ ]:
embeddings

In [ ]:
% ls

build/                  dist/         LICENSE       setup.py
data/                   examples/     LINE_128.csv  temp_struc2vec/
DeepWalk2_128.csv       ge/           pics/
DeepWalk2_64_20_10.csv  ge.egg-info/  README.md


In [ ]:
df = pd.DataFrame.from_dict(embeddings, orient='index')
df.index.name = 'authorID'
df.to_csv('LINE_128.csv')

## Node2Vec 

In [ ]:
from ge import Node2Vec
model = Node2Vec(G, walk_length = walk_length, num_walks = n_walks,p = 0.25, q = 4, workers = 1)#init model
model.train(window_size = 5, iter = 3)# train model
embeddings = model.get_embeddings()# get embedding vectors

Preprocess transition probs...


In [ ]:
df = pd.DataFrame(embeddings, index = list(G.nodes))
df.index.name = 'authorID'
df.to_csv('Node2Vec_128.csv')